In [1]:
import sys
sys.path
sys.path.append('../service/')

print(sys.executable)

/home/stefan/coding/notebooks/env/bin/python


In [2]:
# standart libs
import random
import copy

# 3rd party libs
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt
import gempy as gp
from gempy.utils import section_utils

# local
import functions as fun

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Init the geo_model

In [3]:
# instantiate the geo_model
geo_model = gp.create_model("BaseModel")

# defautl data
geo_model = gp.init_data(
    geo_model,
    extent=[0, 1, 0, 1, 0, 1],
    resolution=[5, 5, 5]
)

# compile theno function
gp.set_interpolation_data(
    geo_model,
    compile_theano=True,
    theano_optimizer='fast_run',
)

Active grids: ['regular']
Setting kriging parameters to their default values.
Compiling theano function...
Level of Optimization:  fast_run
Device:  cpu
Precision:  float64
Number of faults:  0
Compilation Done!
Kriging values: 
                     values
range              1.73205
$C_o$            0.0714286
drift equations        [3]


In [4]:
# set some series
series_df = pd.DataFrame(columns=['name', 'isfault', 'order_series'])
series_df.loc[2] = { 'order_series': 2, 'name': 'Fault_Series', 'isfault': True }
series_df.loc[0] = { 'order_series': 0, 'name': 'Basement', 'isfault': False }
series_df.loc[1] = { 'order_series': 1, 'name': 'Strat_Series', 'isfault': False }

In [5]:
geo_model.add_series(series_list=['S1', 'S2', 'S3'])

,order_series,BottomRelation,isActive
Default series,1,Erosion,False
S1,2,Erosion,NaN
S2,3,Erosion,NaN
S3,4,Erosion,NaN


In [12]:
fun.update_series(geo_model, series_df)

Fault colors changed. If you do not like this behavior, set change_color to False.


In [ ]:
# remove old state
old_series = geo_model.series.df.index.to_list()
geo_model.add_series(series_list=['TO_DELETE'])
geo_model.delete_series(old_series)

# set new state
series_df.sort_index()
geo_model.add_series(series_list=series_df['name'].to_list())
geo_model.delete_series(['TO_DELETE'])
geo_model.set_is_fault(series_df[series_df['isfault']]['name'])